In [ ]:
# From https://www.kaggle.com/bhavikjain/tps-june-21-eda-models

In [1]:
!pip install tensorflow

In [2]:
!pip install tensorflow-addons

In [3]:
import pandas as pd
import numpy as np

from sklearn.model_selection import StratifiedKFold
from sklearn import preprocessing
from sklearn.metrics import log_loss

import gc
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow.keras.backend as K
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import activations,callbacks
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import initializers

from keras.models import Model

In [4]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

X_dim = np.shape(train)[1] - 2 # id and target
print(X_dim)

75


In [5]:
targets = pd.get_dummies(train['target'])

In [6]:
def custom_metric(y_true, y_pred):
    y_pred = K.clip(y_pred, 1e-15, 1-1e-15)
    loss = K.mean(cce(y_true, y_pred))
    return loss

cce = tf.keras.losses.CategoricalCrossentropy()

es = tf.keras.callbacks.EarlyStopping(
    monitor='val_custom_metric', min_delta=1e-05, patience=5, verbose=0,
    mode='min', baseline=None, restore_best_weights=True)

plateau = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_custom_metric', factor=0.7, patience=2, verbose=0,
    mode='min')

In [7]:
def conv_model():

    conv_inputs = layers.Input(shape = (X_dim))
    embed = layers.Embedding (input_dim = 354, 
                              output_dim = 7,
                              embeddings_regularizer='l2')(conv_inputs)
    embed = layers.Conv1D(12,1,activation = 'relu')(embed)        
    embed = layers.Flatten()(embed)
    hidden = layers.Dropout(0.3)(embed)
    
    hidden = tfa.layers.WeightNormalization(
                layers.Dense(
                units=32,
                activation ='selu',
                kernel_initializer = "lecun_normal"))(hidden)
    
    output = layers.Dropout(0.3)(layers.Concatenate()([embed, hidden]))
    output = tfa.layers.WeightNormalization(
    layers.Dense(
                units = 32,
                activation='relu',
                kernel_initializer = "lecun_normal"))(output) 
    output = layers.Dropout(0.4)(layers.Concatenate()([embed, hidden, output]))
    output = tfa.layers.WeightNormalization(
    layers.Dense(
                units = 32, 
                activation = 'relu',
                kernel_initializer = "lecun_normal"))(output)
    
    conv_outputs = layers.Dense(
                units = 9, 
                activation ='softmax',
                kernel_initializer ="lecun_normal")(output)
    
    model = Model(conv_inputs,conv_outputs)
    
    return model

In [8]:
import datetime;
import time;

def do_run(run_number):
    print(datetime.datetime.now())
    start = time.time()

    oof_NN_a = np.zeros((train.shape[0],9))
    pred_NN_a = np.zeros((test.shape[0],9))

    N_FOLDS = 25
    EPOCH = 60

    skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True)

    for fold, (tr_idx, ts_idx) in enumerate(skf.split(train,train.iloc[:,-1])):
        #print(f"\n ====== TRAINING FOLD {fold} =======")

        X_train = train.iloc[:,1:-1].iloc[tr_idx]
        y_train = targets.iloc[tr_idx]
        X_test = train.iloc[:,1:-1].iloc[ts_idx]
        y_test = targets.iloc[ts_idx]

        K.clear_session()

        #print("\n-----Convolution model Training----\n")
        model_conv = conv_model()

        model_conv.compile(loss='categorical_crossentropy', 
                                optimizer= 'adam', #keras.optimizers.Adam(learning_rate=2e-4), 
                                metrics=custom_metric)
        model_conv.fit(X_train, y_train,
                  batch_size = 256, epochs = EPOCH,
                  validation_data=(X_test, y_test),
                  callbacks=[es, plateau],
                  verbose = 0)

        pred_a = model_conv.predict(X_test) 
        oof_NN_a[ts_idx] += pred_a 
        score_NN_a = log_loss(y_test, pred_a)
        print(f"RUN {run_number} - FOLD {fold} - Score: {score_NN_a}")
        pred_NN_a += model_conv.predict(test.iloc[:,1:]) / N_FOLDS 

    score_a = log_loss(targets, oof_NN_a)
    print(f"=== FINAL SCORE CONVOLUTION MODEL : {score_a} ===")
    print(datetime.datetime.now())
    minutes = (time.time() - start) / 60
    print('minutes', minutes, '\n')
    
    return pred_NN_a
    

In [9]:
n_runs = 11
ensemble = do_run(1)

for i in range(2, n_runs+1):
    for_result = do_run(i)
    ensemble = ensemble + for_result
    
ensemble = ensemble / n_runs

2021-06-23 18:00:14.118530
RUN 1 - FOLD 0 - Score: 1.7378556807730348
RUN 1 - FOLD 1 - Score: 1.7492258611375
RUN 1 - FOLD 2 - Score: 1.7391606853217818
RUN 1 - FOLD 3 - Score: 1.7446606615118216
RUN 1 - FOLD 4 - Score: 1.7407376185506582
RUN 1 - FOLD 5 - Score: 1.7547598089973908
RUN 1 - FOLD 6 - Score: 1.7513291065127123
RUN 1 - FOLD 7 - Score: 1.7499952134916092
RUN 1 - FOLD 8 - Score: 1.7462906645406038
RUN 1 - FOLD 9 - Score: 1.7527686940915883
RUN 1 - FOLD 10 - Score: 1.7536981785474346
RUN 1 - FOLD 11 - Score: 1.7446534423273987
RUN 1 - FOLD 12 - Score: 1.7509805950269801
RUN 1 - FOLD 13 - Score: 1.7506020555426367
RUN 1 - FOLD 14 - Score: 1.7433790092724375
RUN 1 - FOLD 15 - Score: 1.7446199341667816
RUN 1 - FOLD 16 - Score: 1.7417886307215666
RUN 1 - FOLD 17 - Score: 1.7507867722739465
RUN 1 - FOLD 18 - Score: 1.761690969325602
RUN 1 - FOLD 19 - Score: 1.739214804316056
RUN 1 - FOLD 20 - Score: 1.7455982735250146
RUN 1 - FOLD 21 - Score: 1.7373001036844216
RUN 1 - FOLD 22 - Sc

RUN 7 - FOLD 20 - Score: 1.7541397407604382
RUN 7 - FOLD 21 - Score: 1.742202415778069
RUN 7 - FOLD 22 - Score: 1.7575877040694468
RUN 7 - FOLD 23 - Score: 1.7515557773066686
RUN 7 - FOLD 24 - Score: 1.747519810299389
=== FINAL SCORE CONVOLUTION MODEL : 1.7469011913770462 ===
2021-06-23 20:12:19.099254
minutes 18.531035673618316 

2021-06-23 20:12:19.107218
RUN 8 - FOLD 0 - Score: 1.7476581852370874
RUN 8 - FOLD 1 - Score: 1.7410039442684502
RUN 8 - FOLD 2 - Score: 1.7588282244643196
RUN 8 - FOLD 3 - Score: 1.7580316488007084
RUN 8 - FOLD 4 - Score: 1.7395539505173219
RUN 8 - FOLD 5 - Score: 1.7486745656318963
RUN 8 - FOLD 6 - Score: 1.745569689560216
RUN 8 - FOLD 7 - Score: 1.7445015237607295
RUN 8 - FOLD 8 - Score: 1.7555068376171403
RUN 8 - FOLD 9 - Score: 1.7397600476618391
RUN 8 - FOLD 10 - Score: 1.7448728649006224
RUN 8 - FOLD 11 - Score: 1.7482703864718787
RUN 8 - FOLD 12 - Score: 1.7507972963536158
RUN 8 - FOLD 13 - Score: 1.7430526059849654
RUN 8 - FOLD 14 - Score: 1.74908313

In [10]:
submission = pd.read_csv("./sample_submission.csv")
submission['Class_1']=ensemble[:,0]
submission['Class_2']=ensemble[:,1]
submission['Class_3']=ensemble[:,2]
submission['Class_4']=ensemble[:,3]
submission['Class_5']=ensemble[:,4]
submission['Class_6']=ensemble[:,5]
submission['Class_7']=ensemble[:,6]
submission['Class_8']=ensemble[:,7]
submission['Class_9']=ensemble[:,8]

submission.to_csv("submission.csv_11_25_60", index=False)